In [4]:
import numpy as np
import os, sys
import h5py
from optparse import OptionParser 
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn import svm
from joblib import dump, load
from sklearn import metrics
import pandas as pd
from sklearn.model_selection import train_test_split
from joblib import dump, load
import pickle

In [19]:
global model
epochs = 20
nclasses = 6
batchsize = 128
model_path = "../models/svm.checkpoint"

In [5]:
filehandler = open("../data/sample_data/hotel_reservation_sample.pkl","rb")
dat = pickle.load(filehandler)
filehandler.close()
print(type(dat))
print(dat.shape)

<class 'pandas.core.frame.DataFrame'>
(999996, 7)


In [6]:
X = dat.drop(columns = "label")
y = dat.loc[:, "label"]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
def createModel(modelname="svm"):
    from sklearn.linear_model import SGDClassifier
    global model

    # SVM classifier trained online with stochastic gradient descent
    model = SGDClassifier(loss="hinge", penalty="l2")
    if modelname == "log":
        # Logistic Regresion classifier trained online with stochastic gradient descent
        model = SGDClassifier(loss="log", penalty="l2")  
        print("Using Logistic Regression...")
    if modelname == "non-linear-svm":
        model = svm.SVC(kernel='rbf')
        print("Using RBF kernel SVM...")
    else:
        print("Using Hinge Loss SVM...")
        
        
createModel()

Using Hinge Loss SVM...


In [20]:
def train( X_train, y_train, nClasses=2, batchSize=256):
    best_score=0
    X_count = X_train.shape[0]
    batchCount= X_count // batchSize

    j=0
    shuffledRange = range(X_count)
    shuffledX = X_train[shuffledRange,]
    shuffledY = [y_train[i] for i in shuffledRange]

    global model
    for i in range(0, batchCount):  # Iterate over "mini-batches" of 1000 samples each
        j+=1
        y_train_batch = shuffledY[i*batchSize :(i +1)* batchSize]
        X_train_batch = shuffledX[i*batchSize :(i +1)* batchSize,]
        # vectorizer.fit_transform(train_data[i:i + batchSize])
        # Update the classifier with documents in the current mini-batch
        model.partial_fit(X_train_batch, y_train_batch, classes=range(nClasses))
        
for i in range(epochs):
    train(X_train.to_numpy(), y_train.to_numpy(), nclasses, batchsize)
    dump(model, model_path)

In [21]:
score = model.score(X_test, y_test)
print(score)

0.97282


In [22]:
y_pred = model.predict(X_test)
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred, pos_label=2)